# Load Modules

In [1]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
from IPython.display import display
from tqdm import tqdm
%reload_ext autoreload
%autoreload 2

# Load Source list files

In [15]:
## This is for CV-old dataset
source = pd.read_csv('../source_list/cv_new_updated/0-150.csv')
source


FileNotFoundError: [Errno 2] No such file or directory: '../source_list/cv_new_updated/0-150.csv'

In [3]:
source = source[source['num_obs']>0]
source = source.replace(np.nan , np.nan).sort_values('num_obs' , ascending=False).reset_index(drop=True)
display(source)

,Unnamed: 0,type,SRC_ID,A_NAME,A_RA,A_DEC,A_DATABASE_TABLE,A_OBSERVATORY,B_NAME,B_RA,B_DEC,num_obs,XOFFSET_
0,17,CV,CV0018,[HPH2013]_176__________,10.687660,41.255870,M31CFCXO________,CHANDRA______,2CXO_J004245.0+411520_,10.687728,41.255816,127,0.004
1,9,CV,CV0010,[HPH2013]_142__________,10.678150,41.262470,M31CFCXO________,CHANDRA______,2CXO_J004242.7+411544_,10.678273,41.262422,104,0.006
2,16,CV,CV0017,[HPH2013]_169__________,10.684970,41.269520,M31CFCXO________,CHANDRA______,2CXO_J004244.3+411608_,10.684755,41.269000,102,0.033
3,109,CV,CV0110,XMMSL2_J004231.9+411621,10.633182,41.272599,XMMSLEWCLN______,XMM-NEWTON___,2CXO_J004231.1+411621_,10.629760,41.272666,102,0.154
4,20,CV,CV0021,[HPH2013]_205__________,10.702900,41.274060,M31CFCXO________,CHANDRA______,2CXO_J004248.7+411626_,10.702984,41.274022,97,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,78,CV,CV0079,CXOGLB_J002405.3-720421,6.022350,-72.072647,GC47TUCCXO______,CHANDRA______,2CXO_J002405.3-720421_,6.022490,-72.072679,1,0.003
83,60,CV,CV0061,2XMM_J190109.3-220003__,285.288670,-22.001120,XMMSSCLWBS______,XMM-NEWTON___,2CXO_J190109.3-220005_,285.288993,-22.001639,1,0.036
84,61,CV,CV0062,2XMM_J231519.0-591029__,348.829250,-59.174770,XMMSSCLWBS______,XMM-NEWTON___,2CXO_J231518.9-591028_,348.828827,-59.174549,1,0.019
85,62,CV,CV0063,CHI_J162011-5002_1_____,245.027875,-50.033056,CHICAGOCXO______,CHANDRA______,2CXO_J162006.7-500159_,245.027924,-50.033097,1,0.003


# Read Chandra data files 

In [4]:
ch_names = []
ch_ra = []
ch_dec = []
offset = []
src_id_list = [] 
n_obs_list = []
df_combined = pd.DataFrame()
for index in tqdm(range(len(source))):
    s = source['A_NAME'][index]
    df = pd.read_csv('../data/all_data/cv_new_data_all/'+s+'.csv' , index_col='index').sort_values('sepn').reset_index(drop=True)
    #display(df)
    top_match = df.loc[0]
    # Select only the source , which has highest cross-matching 
    # And create an array with this seperation value to be stored in original source list table
    # such that source list table have chandra source name , number of actual obs  and seperation.
    # Select all the obs of this cross match and append in all data file

    # Put actual source name , and give source id and obs id in this data
    df_select = df[df['name']==top_match['name']].reset_index(drop=True)
    #df_select = df_select.drop(columns=['name'])
    #display(df_select.head())
    #break
    n_obs_cur = len(df_select)
    obs_id = ['CV_'+str(index)+'_obs_'+str(i) for i in range(len(df_select))]
    src_id = ['CV_'+str(index)]*len(df_select)
    df_select.insert(0 ,'src_id' , src_id)
    df_select.insert(1 ,'obs_id' , obs_id)
    df_select.insert(2, 'num_obs' , [str(len(df_select))]*len(df_select))
    df_select.insert(3 , 'src_n' , [s]*len(df_select))
    top_row = df_select.iloc[0]
    ch_names.append(top_row['name'])
    ch_ra.append(top_row['ra'])
    ch_dec.append(top_row['dec'])
    offset.append(top_row['sepn'])
    n_obs_list.append(len(df_select))
    src_id_list.append('CV_'+str(index))
    #display(df_select)
    df_combined = df_combined.append(df_select)
source_updated = source.copy().reset_index(drop=True)
source_updated.insert(7 , 'chandra_name' , ch_names)
source_updated.insert(8 , 'ch_ra' , ch_ra)
source_updated.insert(9 , 'ch_dec' , ch_dec)
source_updated.insert(10 , 'xoffset' , offset)
source_updated.insert(11 , 'num_obs_used' , n_obs_list)
source_updated = source.reset_index(drop=True)
display(source_updated)
#display(df_combined)
df_combined = df_combined.reset_index(drop=True)
df_combined = df_combined.set_index('obs_id')
df_combined

100%|██████████| 87/87 [00:05<00:00, 15.40it/s]


,Unnamed: 0,type,SRC_ID,A_NAME,A_RA,A_DEC,A_DATABASE_TABLE,A_OBSERVATORY,B_NAME,B_RA,B_DEC,num_obs,XOFFSET_
0,17,CV,CV0018,[HPH2013]_176__________,10.687660,41.255870,M31CFCXO________,CHANDRA______,2CXO_J004245.0+411520_,10.687728,41.255816,127,0.004
1,9,CV,CV0010,[HPH2013]_142__________,10.678150,41.262470,M31CFCXO________,CHANDRA______,2CXO_J004242.7+411544_,10.678273,41.262422,104,0.006
2,16,CV,CV0017,[HPH2013]_169__________,10.684970,41.269520,M31CFCXO________,CHANDRA______,2CXO_J004244.3+411608_,10.684755,41.269000,102,0.033
3,109,CV,CV0110,XMMSL2_J004231.9+411621,10.633182,41.272599,XMMSLEWCLN______,XMM-NEWTON___,2CXO_J004231.1+411621_,10.629760,41.272666,102,0.154
4,20,CV,CV0021,[HPH2013]_205__________,10.702900,41.274060,M31CFCXO________,CHANDRA______,2CXO_J004248.7+411626_,10.702984,41.274022,97,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,78,CV,CV0079,CXOGLB_J002405.3-720421,6.022350,-72.072647,GC47TUCCXO______,CHANDRA______,2CXO_J002405.3-720421_,6.022490,-72.072679,1,0.003
83,60,CV,CV0061,2XMM_J190109.3-220003__,285.288670,-22.001120,XMMSSCLWBS______,XMM-NEWTON___,2CXO_J190109.3-220005_,285.288993,-22.001639,1,0.036
84,61,CV,CV0062,2XMM_J231519.0-591029__,348.829250,-59.174770,XMMSSCLWBS______,XMM-NEWTON___,2CXO_J231518.9-591028_,348.828827,-59.174549,1,0.019
85,62,CV,CV0063,CHI_J162011-5002_1_____,245.027875,-50.033056,CHICAGOCXO______,CHANDRA______,2CXO_J162006.7-500159_,245.027924,-50.033097,1,0.003


,src_id,num_obs,src_n,sepn,instrument,detect_stack_id,name,ra,dec,obsid,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,33,[HPH2013]_176__________,0.000000,ACIS,acisfJ0042424p411621_001,2CXO J004245.0+411520,10.68773,41.25582,4360,...,NaN,NaN,NaN,NaN,103,0,0,63,0,0
CV_0_obs_1,CV_0,33,[HPH2013]_176__________,0.000000,ACIS,acisfJ0042424p411621_001,2CXO J004245.0+411520,10.68773,41.25582,15328,...,NaN,NaN,NaN,NaN,103,0,0,63,0,0
CV_0_obs_2,CV_0,33,[HPH2013]_176__________,0.000000,ACIS,acisfJ0042424p411621_001,2CXO J004245.0+411520,10.68773,41.25582,15327,...,NaN,NaN,NaN,NaN,103,0,0,63,0,0
CV_0_obs_3,CV_0,33,[HPH2013]_176__________,0.000000,ACIS,acisfJ0042424p411621_001,2CXO J004245.0+411520,10.68773,41.25582,15326,...,NaN,NaN,NaN,NaN,103,0,0,63,0,0
CV_0_obs_4,CV_0,33,[HPH2013]_176__________,0.000000,ACIS,acisfJ0042424p411621_001,2CXO J004245.0+411520,10.68773,41.25582,15325,...,NaN,NaN,NaN,NaN,103,0,0,63,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_82_obs_0,CV_82,1,CXOGLB_J002405.3-720421,0.004347,ACIS,acisfJ1231126m141108_001,2CXO J123111.3-141143,187.79710,-14.19553,15362,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
CV_83_obs_0,CV_83,1,2XMM_J190109.3-220003__,0.000000,ACIS,acisfJ1900580m220552_001,2CXO J190109.3-220005,285.28900,-22.00164,3774,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
CV_84_obs_0,CV_84,1,2XMM_J231519.0-591029__,0.000000,ACIS,acisfJ2315509m590334_001,2CXO J231518.9-591028,348.82880,-59.17455,2037,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0


## Save raw table

In [5]:
df_combined.replace(np.nan , '')
df_combined.to_csv('raw_data/cv_new_raw_data.csv' ,)

## Filter flags

In [6]:
from features import col_to_drop

In [7]:
df = pd.read_csv('raw_data/cv_new_raw_data.csv' , index_col='obs_id')
filter_flags = [
    'pileup_flag',
    'mstr_sat_src_flag',
    'mstr_streak_src_flag',
    'sat_src_flag',
    'streak_src_flag',
    ]
df = df.replace({
    '  NAN' : np.nan , 
    '      NaN' : np.nan , 
    'FALSE' : False , 
    'False' : False , 
    ' TRUE' : True , 
    
    })
print('BEFORE FILTERING')
display(df)
for f in filter_flags:
    df = df[df[f]==False]
print('AFTER FILTERING')
#display(df)
df = df.drop(columns=col_to_drop)
df

BEFORE FILTERING


,src_id,num_obs,src_n,sepn,instrument,detect_stack_id,name,ra,dec,obsid,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,33,[HPH2013]_176__________,0.000000,ACIS,acisfJ0042424p411621_001,2CXO J004245.0+411520,10.68773,41.25582,4360,...,NaN,NaN,NaN,NaN,103,0,0,63,0,0
CV_0_obs_1,CV_0,33,[HPH2013]_176__________,0.000000,ACIS,acisfJ0042424p411621_001,2CXO J004245.0+411520,10.68773,41.25582,15328,...,NaN,NaN,NaN,NaN,103,0,0,63,0,0
CV_0_obs_2,CV_0,33,[HPH2013]_176__________,0.000000,ACIS,acisfJ0042424p411621_001,2CXO J004245.0+411520,10.68773,41.25582,15327,...,NaN,NaN,NaN,NaN,103,0,0,63,0,0
CV_0_obs_3,CV_0,33,[HPH2013]_176__________,0.000000,ACIS,acisfJ0042424p411621_001,2CXO J004245.0+411520,10.68773,41.25582,15326,...,NaN,NaN,NaN,NaN,103,0,0,63,0,0
CV_0_obs_4,CV_0,33,[HPH2013]_176__________,0.000000,ACIS,acisfJ0042424p411621_001,2CXO J004245.0+411520,10.68773,41.25582,15325,...,NaN,NaN,NaN,NaN,103,0,0,63,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_82_obs_0,CV_82,1,CXOGLB_J002405.3-720421,0.004347,ACIS,acisfJ1231126m141108_001,2CXO J123111.3-141143,187.79710,-14.19553,15362,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
CV_83_obs_0,CV_83,1,2XMM_J190109.3-220003__,0.000000,ACIS,acisfJ1900580m220552_001,2CXO J190109.3-220005,285.28900,-22.00164,3774,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
CV_84_obs_0,CV_84,1,2XMM_J231519.0-591029__,0.000000,ACIS,acisfJ2315509m590334_001,2CXO J231518.9-591028,348.82880,-59.17455,2037,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0


AFTER FILTERING


,src_id,num_obs,src_n,sepn,name,ra,dec,photflux_aper_s,photflux_aper_lolim_s,photflux_aper_hilim_s,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,33,[HPH2013]_176__________,0.000000,2CXO J004245.0+411520,10.68773,41.25582,0.0,0.0,7.766e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_1,CV_0,33,[HPH2013]_176__________,0.000000,2CXO J004245.0+411520,10.68773,41.25582,0.0,0.0,1.703e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_2,CV_0,33,[HPH2013]_176__________,0.000000,2CXO J004245.0+411520,10.68773,41.25582,1.555e-06,0.0,3.109e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_3,CV_0,33,[HPH2013]_176__________,0.000000,2CXO J004245.0+411520,10.68773,41.25582,0.0,0.0,1.4919999999999999e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_4,CV_0,33,[HPH2013]_176__________,0.000000,2CXO J004245.0+411520,10.68773,41.25582,0.0,0.0,1.476e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_82_obs_0,CV_82,1,CXOGLB_J002405.3-720421,0.004347,2CXO J123111.3-141143,187.79710,-14.19553,3.896e-05,3.395e-05,4.397e-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_83_obs_0,CV_83,1,2XMM_J190109.3-220003__,0.000000,2CXO J190109.3-220005,285.28900,-22.00164,0.00032450000000000003,0.0003102,0.000338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_84_obs_0,CV_84,1,2XMM_J231519.0-591029__,0.000000,2CXO J231518.9-591028,348.82880,-59.17455,5.855e-06,5.026e-06,6.682999999999999e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
from features import arranged_cols  , id_cols 

In [9]:
df_id = df[id_cols]
df_val = df.drop(columns=id_cols)
val_col_list = df_val.columns.to_list()
for v in val_col_list:
    df_val.loc[:,v] = pd.to_numeric(df_val.loc[:,v])
df_val

,photflux_aper_s,photflux_aper_lolim_s,photflux_aper_hilim_s,flux_aper_s,flux_aper_lolim_s,flux_aper_hilim_s,streak_src_flag,likelihood_b,likelihood_h,likelihood_m,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,0.000000,0.000000,7.766000e-07,0.000000e+00,0.000000e+00,1.375000e-15,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_1,0.000000,0.000000,1.703000e-06,0.000000e+00,0.000000e+00,2.319000e-15,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_2,0.000002,0.000000,3.109000e-06,2.936000e-15,1.727000e-16,5.699000e-15,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_3,0.000000,0.000000,1.492000e-06,0.000000e+00,0.000000e+00,2.031000e-15,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_4,0.000000,0.000000,1.476000e-06,0.000000e+00,0.000000e+00,2.011000e-15,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_82_obs_0,0.000039,0.000034,4.397000e-05,1.095000e-13,9.545000e-14,1.236000e-13,False,634.48,51.36,218.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_83_obs_0,0.000325,0.000310,3.380000e-04,5.641000e-13,5.392000e-13,5.876000e-13,False,7292.73,762.11,2011.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_84_obs_0,0.000006,0.000005,6.683000e-06,1.187000e-14,1.015000e-14,1.359000e-14,False,301.47,21.27,165.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_ok = pd.concat([df_id , df_val] , axis=1)
df_ok

,src_id,num_obs,sepn,name,src_n,ra,dec,livetime,significance,likelihood,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,33,0.000000,2CXO J004245.0+411520,[HPH2013]_176__________,10.68773,41.25582,4903.5,11.31,304.4646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_1,CV_0,33,0.000000,2CXO J004245.0+411520,[HPH2013]_176__________,10.68773,41.25582,3974.2,11.31,304.4646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_2,CV_0,33,0.000000,2CXO J004245.0+411520,[HPH2013]_176__________,10.68773,41.25582,3907.5,11.31,304.4646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_3,CV_0,33,0.000000,2CXO J004245.0+411520,[HPH2013]_176__________,10.68773,41.25582,3907.5,11.31,304.4646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_4,CV_0,33,0.000000,2CXO J004245.0+411520,[HPH2013]_176__________,10.68773,41.25582,3907.5,11.31,304.4646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_82_obs_0,CV_82,1,0.004347,2CXO J123111.3-141143,CXOGLB_J002405.3-720421,187.79710,-14.19553,9936.3,9.99,634.4813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_83_obs_0,CV_83,1,0.000000,2CXO J190109.3-220005,2XMM_J190109.3-220003__,285.28900,-22.00164,5049.0,31.89,7292.7310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_84_obs_0,CV_84,1,0.000000,2CXO J231518.9-591028,2XMM_J231519.0-591029__,348.82880,-59.17455,49313.1,10.04,301.4733,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
from features import phot_flux , phot_flux_hilim , phot_flux_lolim , en_flux , en_flux_hilim , en_flux_lolim

In [12]:
flux_cols =  en_flux + en_flux_hilim + en_flux_lolim 

In [13]:

df_flux_filt = df_ok.copy().replace(0 , np.nan).replace(np.nan , -9987)
df_flux_filt.iloc[:, 10:]
for f in flux_cols:
    df_flux_filt = df_flux_filt.query(f+'<1e-12')
df_flux_filt = df_flux_filt.replace(-9987 , np.nan)
df_flux_filt = df_flux_filt.query('likelihood > 50')
df_flux_filt = df_flux_filt.query('significance > 3')
df_flux_filt

,src_id,num_obs,sepn,name,src_n,ra,dec,livetime,significance,likelihood,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,33,NaN,2CXO J004245.0+411520,[HPH2013]_176__________,10.68773,41.25582,4903.5,11.31,304.4646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_1,CV_0,33,NaN,2CXO J004245.0+411520,[HPH2013]_176__________,10.68773,41.25582,3974.2,11.31,304.4646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_2,CV_0,33,NaN,2CXO J004245.0+411520,[HPH2013]_176__________,10.68773,41.25582,3907.5,11.31,304.4646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_3,CV_0,33,NaN,2CXO J004245.0+411520,[HPH2013]_176__________,10.68773,41.25582,3907.5,11.31,304.4646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_0_obs_4,CV_0,33,NaN,2CXO J004245.0+411520,[HPH2013]_176__________,10.68773,41.25582,3907.5,11.31,304.4646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_78_obs_0,CV_78,1,NaN,2CXO J063344.1+063230,2E_1936.6+3023_________,98.43393,6.54178,19917.4,14.54,1398.9470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_82_obs_0,CV_82,1,0.004347,2CXO J123111.3-141143,CXOGLB_J002405.3-720421,187.79710,-14.19553,9936.3,9.99,634.4813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_84_obs_0,CV_84,1,NaN,2CXO J231518.9-591028,2XMM_J231519.0-591029__,348.82880,-59.17455,49313.1,10.04,301.4733,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_flux_filt.to_csv('filtered_data/cv_new_data.csv')